# Deploying A Sentence-Embedding Model

## Contents

1. [Setup](#Setup)
2. [Deploy](#Deploy)
3. [Evaluate](#Evaluate)
4. [Update](#UPDATE)

# Setup

- Load environmental variables
- Create Sagemaker Session
- Define S3 Bucket, path and model name
- Provide IAM role

In [ ]:
%load_ext dotenv
import json
import os
import sagemaker
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorchModel

%dotenv -v

sagemaker_session = sagemaker.Session()

bucket = os.getenv("S3_BUCKET")
model_location = f"{bucket}{os.getenv('MODEL_PATH')}"               # path to model data
entry = os.getenv("ENTRY_POINT")                                    # Python file to execute
source = os.getenv("SOURCE_DIR")                                    # Source for code
version = os.getenv("FRAMEWORK")                                    # Pytorch Version
role = sagemaker.get_execution_role()                               # IAM role
py_ver = os.getenv("PY_VER")

# git_conf = {'repo': 'https://github.com/sam3ay/sentence-compare.git', 'branch': 'code'}
git_config = {'repo': os.getenv("REPO"), 'branch': os.getenv("BRANCH")}

# Deploy

Provide custom hosting functions:
- input_fn: Deserializes input data
- model_fn: Loads pytorch model
- predict_fn: run inference with input data
- output_fn: Serializes results of predict_fn
- predictor_cls: Creates the predictor with the endpoint name

In [ ]:
class JSONPredictor(RealTimePredictor):
    """Serializes and deserializes json
    """
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [ ]:
model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version=version,
                     entry_point=entry,
                     source_dir=source,
                     predictor_cls=JSONPredictor,
                     git_config=git_config,
                     py_version="py3")

In [ ]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name="sentence-similarity")

# Evaluate

Determine Textual Similarity

In [ ]:
address = {
    "query": ["Speak to people in a formal way."],
    "corpus": [
        "the particulars of the place where someone lives or an organization is situated.",
        "a formal speech delivered to an audience.",
        "write the name and address of the intended recipient on (an envelope, letter, or package).",
        "speak to (a person or an assembly), typically in a formal way.",
    ],
}

response = predictor.predict(address)
print(response)

In [ ]:
sagemaker_session.delete_endpoint(predictor.endpoint)